# 🧭 Discovering and Downloading Google Buildings V3 Tiles for a Region of Interest

## 📋 Overview

In this notebook, you will:

- Load the Google V3 tile index file
- Load your Region of Interest (ROI) polygons (can be single or multiple disjoint areas)
- Filter tiles that intersect with the ROI
- Download only those tiles (as `.geojson.gz`) to a local folder

## 📦 Required Libraries

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
from tqdm import tqdm

## 🔹 Step 1: Load the Google Buildings V3 Tile Index

In [ ]:
# Path to the official tile index GeoJSON file (public metadata) - provided in github repo
tile_path = r"/home/saiganeshv/Desktop/Google V3 Download/tiles.geojson"

# Load the tile index as a GeoDataFrame
tiles_gdf = gpd.read_file(tile_path)

# Preview the tile index
print(tiles_gdf.head())

# View column names
print("Columns:", tiles_gdf.columns.tolist())

# Check CRS (should be WGS84 - EPSG:4326)
print("CRS:", tiles_gdf.crs)


### 🗺️ Visualize All Available Tiles

In [ ]:
tiles_gdf.plot(figsize=(12, 10), edgecolor='black')
plt.title("Google Buildings V3 – Global Tile Coverage")
plt.show()

## 🔹 Step 2: Load Your Region of Interest (ROI)

This can be any polygon: city boundary, country grid, slum clusters, etc.

In [ ]:
# Path to your region of interest (ROI) polygons
grid_fp = r"/home/saiganeshv/Desktop/Google V3 Download/sample_ROI.gpkg"

# Load the GeoPackage file (use your ROI source here)
country_grid = gpd.read_file(grid_fp)

# Ensure the CRS matches the tiles CRS
if country_grid.crs != tiles_gdf.crs:
    country_grid = country_grid.to_crs(tiles_gdf.crs)


### 🧭 Convert ROI to a Single Geometry for Efficient Filtering

In [ ]:
# Merge all ROI polygons into one (multi-part geometry)
roi_geom = country_grid.unary_union

## 🔹 Step 3: Filter Only the Tiles That Intersect with ROI

In [ ]:
# Select only tiles that intersect with the ROI and are non-empty
tiles_subset = tiles_gdf[
    tiles_gdf.intersects(roi_geom) & 
    (tiles_gdf["size_mb"] > 0)
]

print(f"Tiles selected for download: {len(tiles_subset)}")

### 🗺️ Visualize Selected Tiles

In [ ]:
tiles_subset.plot(edgecolor='red', figsize=(10, 8))
plt.title("Tiles Intersecting ROI")
plt.show()

## 🔹 Step 4: Download the Filtered Tiles

In [ ]:
# Output folder to save downloaded tiles
out_dir = r"/home/saiganeshv/Desktop/Google V3 Download/Outputs/sample_ROI_tiles"
os.makedirs(out_dir, exist_ok=True)

In [ ]:
# Loop through each selected tile and download only if not already present
for _, row in tqdm(tiles_subset.iterrows(), total=len(tiles_subset)):
    url = row["tile_url"]
    tile_id = row["tile_id"]
    out_path = os.path.join(out_dir, f"{tile_id}.geojson.gz")

    if not os.path.exists(out_path):
        try:
            r = requests.get(url, timeout=60)
            r.raise_for_status()  # Raise error for failed requests
            with open(out_path, "wb") as f:
                f.write(r.content)
        except Exception as e:
            print(f"❌ Failed to download {tile_id}: {e}")
    else:
        print(f"✔️ Already exists: {tile_id}")


### ✅ Notebook Summary

At the end of this notebook, you will have:
- A filtered list of tiles that intersect with your area of interest
- A local folder containing only the relevant `.geojson.gz` files
- This prepares you for efficient processing in Step 2, where you'll clip and extract only relevant building footprints

*Note: The `tiles.geojson` file is provided in github repo*

**If you want to get notified when the download is completed or Jupyter Notebook is done running, get a notification on Telegram. Look at this blog post:** https://sola.kau.se/deprimap/2025/07/16/telegram-bot/